In [2]:
print("hello")

hello


In [3]:
import numpy as np
from numpy import pi, log


#some constants
h = 0.678 #Gaia 2016

G = 6.67430e-11

Msun = 1.98847e30
kpc = 3.0856775814913673e19


rhocrit = 3*(100*h/(kpc))**2/(8*np.pi*G) * kpc**3 / Msun

#NFW  density
def rhoNFW(r,rhos,rs):
    return rhos/((r/rs)*(1.+(r/rs))**2.)


fac = 102

def delc(c):
    return (fac/3)*c**3/(log(1+c)-c/(1+c))
    
#NFW density (virial mass in Msun, concentration parameter)
def rho_nfw(r, mvir, cvir):
    
    rv = (mvir/((4/3) * pi * fac * rhocrit))**(1/3) 
    
    rs = rv / cvir
    
    rhos = rhocrit*delc(cvir)
    
    return rhoNFW(r, rhos, rs)

#Enclosed mass (virial mass in Msun, concentration parameter) in Msun
def mass_nfw(r, mvir, cvir):
    
    rv = (mvir/((4/3) * pi * fac * rhocrit))**(1/3)
    
    rs = rv / cvir
    
    rhos = rhocrit*delc(cvir)
    
    return 4 * pi *  rhos * rs**3 * (log((r + rs) / rs) + rs/(r + rs) - 1) 

In [4]:
import numpy as np
from numpy import pi, log

In [5]:
class Galaxy():
    """
    Derive a dark matter density profile from a galaxy rotation curve

    Args:
        mvir (float): galaxy virial mass (Msun)
        cvir (float): dark matter concentration parameter (Msun)
    """

    def __init__(self, mvir, cvir, data_vel, data_err, data_r):

        #Define constants
        self.h = 0.678 #Gaia 2016
        self.G = 6.67430e-11
        self.Msun = 1.98847e30
        self.kpc = 3.0856775814913673e19
        self.fac = 102

        self.rhocrit = 3*(100*self.h/(self.kpc))**2/(8*np.pi*self.G) * self.kpc**3 / self.Msun

        # Data arrays
        self.data_vel = data_vel # Velocity
        self.data_err = data_err # Velocity errors
        self.data_r = data_r # Radii

        self.mvir = mvir
        self.cvir = cvir

    def delc(self):
        return (self.fac/3)*self.cvir**3/(log(1+self.cvir)-self.cvir/(1+self.cvir))

    def rho_nfw(self, r):
        #NFW density (virial mass in Msun, concentration parameter)
    
        rv = (self.mvir/((4/3) * np.pi * self.fac * self.rhocrit))**(1/3) 
        rs = rv / self.cvir
        rhos = self.rhocrit*self.delc()
    
        return rhos/((r/rs)*(1.+(r/rs))**2.)

    def mass_nfw(self, r):
        #Enclosed mass (virial mass in Msun, concentration parameter) in Msun
    
        rv = (self.mvir/((4/3) * np.pi * self.fac * self.rhocrit))**(1/3)
        rs = rv / self.cvir
        rhos = self.rhocrit*self.delc()
    
        return 4 * np.pi * rhos * rs**3 * (log((r + rs) / rs) + rs/(r + rs) - 1) 
    
    def log_like(self, c, log_m_h):
        '''
        Calculate log-likelihood using the data and NFW parameters (c, log10(M_h))
        '''

        # Calculate enclosed mass array with NFW profile
        enclosed_mass = mass_nfw(self.data_r, 10**log_m_h, c)
        # Calculate predicted velocity from v^2 = GM/r
        pred_vel = np.sqrt(self.G * enclosed_mass / self.data_err)

        # Calculate log-likelihood
        llh = -1/2 * np.sum(((self.data_vel - pred_vel) / self.data_err) ** 2)

        return llh